In [1]:
import pandas as pd
from glob import glob
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
ROOT = "evals"

In [3]:
ls evals/ghs_1/

report/


In [4]:
numbers, genes =[], []
for file in glob("{}/*/report/*.csv".format(ROOT)):
    base_name = file.split("/")[1]
    family, evalue = base_name.split("_")
    if "numbers_clean" in file:
        number = pd.read_csv(file)
        number.loc[:, 'family'] = family
        number.loc[:, 'evalue'] = evalue
        numbers.append(number)
    elif "genes" in file:
        gene = pd.read_csv(file)
        gene.loc[:, "family"] = family
        gene.loc[:, "Evalue"] = evalue
        genes.append(gene)
numbers_table = pd.concat(numbers)
genes_table = pd.concat(genes)

In [5]:
number_group = numbers_table.groupby(['species','family','evalue'])

In [6]:
total_paralogs = number_group.sum()['paralogs']

In [7]:
number_clusters = number_group.count()['cluster']

In [8]:
cluster_paralogs = pd.concat([total_paralogs, number_clusters],1)

In [43]:
for sp in set(cluster_paralogs.index.get_level_values(0)):
    print(sp)
    for value_plot in ['cluster', 'paralogs']:
        sp_title = sp.replace("_", " ")
        sp_tab = cluster_paralogs.loc[sp]
        evalues = ['1','1e-1','1e-2','1e-3','1e-4','1e-5','1e-6','1e-7','1e-8','1e-9','1e-10']
        barplot = sns.barplot(x="family",y=value_plot,hue='evalue',data=sp_tab.reset_index(),
                              hue_order=evalues, palette=sns.color_palette("gray",n_colors=11))
        barplot.legend(loc=(1,0),title='Evalue',ncol=1)
        plt.title(sp_title)
        plt.ylabel(value_plot)
        plt.setp(barplot.get_xticklabels(), rotation=90);
        plt.tight_layout(rect=(0,0,0.85,1))
        plt.savefig("/home/alekos/Dropbox/placenta_clusters_paper/revision/figures/{}_{}.svg".format(sp,value_plot))
        plt.close()

Macaca_fascicularis
Pan_troglodytes
Sus_scrofa
Oryctolagus_cuniculus
Rattus_norvegicus
Gorilla_gorilla_gorilla
Monodelphis_domestica
Ovis_aries
Felis_catus
Chlorocebus_sabaeus
Callithrix_jacchus
Mus_musculus
Nomascus_leucogenys
Equus_caballus
Papio_anubis
Capra_hircus
Bos_taurus
Macaca_mulatta
Canis_lupus_familiaris
Pongo_abelii
Ornithorhynchus_anatinus
Microtus_ochrogaster
Homo_sapiens


In [10]:
rat_1 = numbers_table.loc[(numbers_table['species']=='Rattus_norvegicus')&
                          (numbers_table['family']=="siglecs")].sort_values("family")

In [27]:
cluster_paralogs.loc[('Bos_taurus','globin')]

,paralogs,cluster
evalue,,
1,11,2
1e-1,11,2
1e-2,11,2
1e-3,11,2


In [12]:
rat_siglecs = genes_table.loc[(genes_table['species']=="Rattus_norvegicus")&
                              (genes_table['family']=='siglecs')&
                              (genes_table['Evalue'].isin(['1','1e-1']))&
                              ~(genes_table['cluster'].isin(["na","0.0"]))]

In [13]:
diff_genes = set(rat_siglecs.loc[rat_siglecs['Evalue']=="1e-1","symbol"]) - set(rat_siglecs.loc[rat_siglecs['Evalue']=="1","symbol"])

In [14]:
rat_diff = rat_siglecs.loc[rat_siglecs['symbol'].isin(diff_genes)]

In [15]:
rat_diff

,query,species,chromosome,prot_acc,symbol,start,end,strand,evalue,Length,cluster,order,family,Evalue


In [16]:
rat_1.loc[rat_1['cluster'].isin(rat_diff['cluster'])]

,species,chromosome,cluster,start,end,paralogs,total_genes,proportion_paralogs,perc95_gw,family,evalue


In [17]:
!rm -r evals/siglecs_1e-1/

rm: cannot remove 'evals/siglecs_1e-1/': No such file or directory
